In [1]:
import pandas as pd
import pickle
import sys
from add_features import add_features
import import_ipynb
from feeds import BackFeed,DataFeed
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import os
from openai_utils import generate, count_tokens
from abc import ABC, abstractmethod
from typing import Union, List, Tuple, Dict

importing Jupyter notebook from feeds.ipynb
importing Jupyter notebook from synfeed.ipynb
importing Jupyter notebook from india_calendar.ipynb
importing Jupyter notebook from featfuncs.ipynb


In [2]:
feed = pickle.load(open('../algodata/realdata/datafeed_alldata_True_True_False.pkl', 'rb'))

In [5]:
with open('additional_utils/cols.pkl', 'rb') as f:
        d = pickle.load(f)
        
# d['discrete_cols']+d['pre_discrete_cols']
imp_cols = d['imp_cols']
cols_to_use = d['cols_to_use']
col_counts = d['col_counts']
col_counts = {k:v for k,v in sorted(col_counts.items(), key=lambda x:-x[1])}
print(col_counts)

{'Open-Close_ma_1-Open-Close_ma_10_val': 5, 'Volume_ma_10_val': 4, 'Close_slope_1_polarity': 3, 'Open-Close_ma_1-Open-Close_ma_50_val': 3, 'BBP_5_2.0_slope_1_changelen_val': 3, 'Open-Close_slope_1_val': 2, 'BBM_5_2.0_ma_1-BBM_5_2.0_ma_20_changelen_val': 2, 'BBL_5_2.0_ma_10_slope_15_val': 2, 'MACDs_12_26_9_ma_20_changelen_val': 1, 'Open-High_ma_5-Open-High_ma_20_changelen_val': 1, 'BBL_5_2.0_ma_5_slope_1_changelen_polarity': 1, 'High_n-Low_n_ma_50_changelen_val': 1, 'High_n-Low_n_ma_20_slope_15_changelen_val': 1, 'Open_n-Close_n_ma_10_val': 1, 'Volume_ma_5_slope_5_val': 1, 'Close_n_slope_1_polarity': 1, 'Low_ma_5_slope_3_polarity': 1, 'Open_n_ma_10_slope_5_changelen_polarity': 1, 'RSI_14_ma_1_val': 1, 'Open-Close_polarity': 1, 'Open-High_ma_1-Open-High_ma_50_val': 1, 'Open-Close_ma_1-Open-Close_ma_20_val': 1, 'Close_n_ma_1-Close_n_ma_50_changelen_polarity': 1, 'High_n_ma_1-High_n_ma_20_changelen_polarity': 1, 'Open-High_ma_1-Open-High_ma_50_polarity': 1}


In [6]:
NUM_FEATURES = 5

features = list(col_counts.keys())[:NUM_FEATURES]
print(features)

['Open-Close_ma_1-Open-Close_ma_10_val', 'Volume_ma_10_val', 'Close_slope_1_polarity', 'Open-Close_ma_1-Open-Close_ma_50_val', 'BBP_5_2.0_slope_1_changelen_val']


In [7]:
feature_mapper = {
    'val': {0: 'low', 1: 'medium', 2: 'high'},
    'polarity': {-1: 'negative', 1: 'positive'}
}

In [8]:
ticker = list(feed.data.keys())[0]
df = feed.data[ticker]
print(ticker, df.shape)
df.head()

NTPC.NS (10210, 52)


,row_num,Open_n,High_n,Low_n,Close_n,Volume_n,SMA_10,SMA_20,VOL_SMA_20,RSI_14,...,Open-Close_ma_1-Open-Close_ma_20_val,Close_n_ma_1-Close_n_ma_50_changelen_polarity,High_n_ma_1-High_n_ma_20_changelen_polarity,Open-High_ma_1-Open-High_ma_50_polarity,Date,datetime,Open,Close,High,Low
datetime,,,,,,,,,,,,,,,,,,,,,
2022-02-02 09:15:00+05:30,0,0.997189,1.002108,0.995784,1.001757,1.0,0.994027,0.993693,189623.85,66.498059,...,1,-1,-1,-1,02-Feb-2022,2022-02-02 09:15:00+05:30,141.899994,142.550003,142.600006,141.699997
2022-02-02 09:20:00+05:30,1,1.002460,1.002460,0.998946,0.999649,1.0,0.994483,0.994290,183098.85,60.431660,...,2,-1,1,1,02-Feb-2022,2022-02-02 09:20:00+05:30,142.649994,142.250000,142.649994,142.149994
2022-02-02 09:25:00+05:30,2,1.000000,1.000351,0.995081,0.995081,1.0,0.994554,0.994483,179218.20,49.825826,...,2,-1,-1,1,02-Feb-2022,2022-02-02 09:25:00+05:30,142.300003,141.600006,142.350006,141.600006
2022-02-02 09:30:00+05:30,3,0.995081,0.995081,0.993324,0.993675,1.0,0.994624,0.994571,182077.85,47.087305,...,1,-1,-1,1,02-Feb-2022,2022-02-02 09:30:00+05:30,141.600006,141.399994,141.600006,141.350006
2022-02-02 09:35:00+05:30,4,0.994027,0.994729,0.992270,0.994378,1.0,0.994659,0.994571,181169.20,48.608237,...,0,1,1,1,02-Feb-2022,2022-02-02 09:35:00+05:30,141.449997,141.500000,141.550003,141.199997


In [29]:
class Symbol(ABC):
    @abstractmethod
    def convert(self, data: pd.DataFrame, **kwargs) -> pd.DataFrame:
        pass

In [40]:
class CandleSymbol(Symbol):
    candle_abbreviations = {'R': 'Red', 'G': 'Green', 'H': 'High', 'M': 'Medium', 'L': 'Low'}

    
    def convert(self, data:pd.DataFrame, **kwargs)->pd.DataFrame:
        expand = kwargs.get('expand', True)
        separator = kwargs.get('separator', '-')
        
        data['symbol'] = ''
        open, close = data['Open'], data['Close']
        green = (close - open)>0
        data.loc[green, 'symbol'] = 'G' if not expand else CandleSymbol.candle_abbreviations['G']
        data.loc[~green, 'symbol'] = 'R' if  not expand else CandleSymbol.candle_abbreviations['R']
        
        data.loc[:, 'symbol'] += separator

        diff = np.abs((close-open).values)
        low_thresh, high_thresh = np.quantile(diff, 0.25), np.quantile(diff, 0.75)
        
        data.loc[(diff <= low_thresh), 'symbol'] += 'L' if not expand else CandleSymbol.candle_abbreviations['L']
        data.loc[(diff > low_thresh) & (diff < high_thresh), 'symbol'] += 'M' if not expand else CandleSymbol.candle_abbreviations['M']
        data.loc[(diff >= high_thresh), 'symbol'] += 'H' if not expand else CandleSymbol.candle_abbreviations['H']
        return data

In [26]:
class PromptOrder(ABC):
    @abstractmethod
    def order(self, prompt: Union[str, List], **kwargs) -> str:
        pass

In [21]:
class SameOrder(PromptOrder):
    def order(self, prompt: str | List, **kwargs) -> str:
        if type(prompt) == list:
            return "\n".join(prompt)

In [81]:
class RandomOrder(PromptOrder):
    def order(self, prompt: str | List, **kwargs) -> str:
        if type(prompt) == str:
            prompt_list = prompt.split("\n")
        else:
            prompt_list = prompt
            
        np.random.shuffle(prompt_list)
        return "\n".join(prompt_list)

In [23]:
class DistanceBasedOrder(PromptOrder):
    def order(self, prompt: str | List, **kwargs) -> str:
        assert "distance" in kwargs
        distance = kwargs["distance"]
        closest_last = kwargs.get("closest_last", True)
        
        if type(prompt) == str:
            prompt_list = prompt.split("\n")
        else:
            prompt_list = prompt
            
        assert len(distance) == len(prompt_list)
        prompts_distance = list(zip(prompt_list, distance))
        
        prompts_distance.sort(key=lambda x: x[1], reverse=closest_last)
        prompt_list = [x[0] for x in prompts_distance]
        
        prompt = "\n".join(prompt_list)
        return prompt

In [68]:
class PromptSelection(ABC):
    @abstractmethod
    def select(self, sample:str, all_samples:Dict, num_select:int=5, **kwargs) -> List:
        pass

In [69]:
class SimilarPrompts(PromptSelection):
    def select(self, sample: str, all_samples: Dict, num_select: int = 5, **kwargs) -> List:
        return []

In [135]:
class RecentPrompts(PromptSelection):
    def select(self, sample: str, all_samples: Dict, num_select: int = 5, **kwargs) -> List:
        return []

In [137]:
import numpy as np

class InitialPrompts(PromptSelection):
    def select(self, sample: str, all_samples: Dict, num_select: int = 5, **kwargs) -> List:
        all_samples = {k: v for k, v in all_samples.items() if v is not None}
        idxs_to_chose = list(all_samples.keys())
        return idxs_to_chose[:num_select]

In [138]:
import numpy as np

class RandomPrompts(PromptSelection):
    def select(self, sample: str, all_samples: Dict, num_select: int = 5, **kwargs) -> List:
        all_samples = {k: v for k, v in all_samples.items() if v is not None}
        idxs_to_chose = list(all_samples.keys())
        return np.random.choice(idxs_to_chose, num_select, replace=False)

In [97]:
class FeatureString:
    def __init__(self, features):
        self.features = features
        self.feature_mapper = {'val': {0: 'low', 1: 'medium', 2: 'high'}, 'polarity': {-1: 'negative', 1: 'positive'}}
        
    def convert(self, feature_list):
        f_list = []
        for idx, feature in enumerate(self.features):
            for key in self.feature_mapper:
                if feature.endswith(key):
                    f_list.append(f'f{idx+1}: {self.feature_mapper[key][feature_list[idx]]}')
                    
        feature_str = ', '.join(f_list)
        return feature_str

In [98]:
def get_current(df, idx, feature_converter:FeatureString, last_k=5):
    num = df.iloc[idx, :]['row_num']
    if num < last_k: return None
    
    prev_5 = df.iloc[idx-last_k:idx, :]['symbol'].values
    prev_5_str = ', '.join(prev_5)
    
    feature_vals = df.iloc[idx][feature_converter.features].values.tolist()
    feature_str = feature_converter.convert(feature_vals)
        
    complete_str = f'Previous Candles: {prev_5_str} Features: {feature_str}'
    return complete_str

In [100]:
def get_future(df, idx, next_k=5):
    num = df.iloc[idx, :]['row_num']
    if num + next_k > 75: return None
    
    next_5 = df.iloc[idx:idx+next_k+1, :]['symbol'].values
    next_5_str = f'Current Candle: {next_5[0]} Next: {", ".join(next_5[1:])}'
    
    return next_5_str

In [173]:
symbol_mapper = CandleSymbol()
feature_converter = FeatureString(features=features)
prompt_orderer = RandomOrder()
prompt_selector = InitialPrompts()

In [102]:
df['row_num'] = df['row_num'].apply(lambda x: x-75 if x>74 else x)

In [103]:
df = symbol_mapper.convert(df, expand=True)
df.head()

,row_num,Open_n,High_n,Low_n,Close_n,Volume_n,SMA_10,SMA_20,VOL_SMA_20,RSI_14,...,Close_n_ma_1-Close_n_ma_50_changelen_polarity,High_n_ma_1-High_n_ma_20_changelen_polarity,Open-High_ma_1-Open-High_ma_50_polarity,Date,datetime,Open,Close,High,Low,symbol
datetime,,,,,,,,,,,,,,,,,,,,,
2022-02-02 09:15:00+05:30,0,0.997189,1.002108,0.995784,1.001757,1.0,0.994027,0.993693,189623.85,66.498059,...,-1,-1,-1,02-Feb-2022,2022-02-02 09:15:00+05:30,141.899994,142.550003,142.600006,141.699997,Green-High
2022-02-02 09:20:00+05:30,1,1.002460,1.002460,0.998946,0.999649,1.0,0.994483,0.994290,183098.85,60.431660,...,-1,1,1,02-Feb-2022,2022-02-02 09:20:00+05:30,142.649994,142.250000,142.649994,142.149994,Red-High
2022-02-02 09:25:00+05:30,2,1.000000,1.000351,0.995081,0.995081,1.0,0.994554,0.994483,179218.20,49.825826,...,-1,-1,1,02-Feb-2022,2022-02-02 09:25:00+05:30,142.300003,141.600006,142.350006,141.600006,Red-High
2022-02-02 09:30:00+05:30,3,0.995081,0.995081,0.993324,0.993675,1.0,0.994624,0.994571,182077.85,47.087305,...,-1,-1,1,02-Feb-2022,2022-02-02 09:30:00+05:30,141.600006,141.399994,141.600006,141.350006,Red-Medium
2022-02-02 09:35:00+05:30,4,0.994027,0.994729,0.992270,0.994378,1.0,0.994659,0.994571,181169.20,48.608237,...,1,1,1,02-Feb-2022,2022-02-02 09:35:00+05:30,141.449997,141.500000,141.550003,141.199997,Green-Low


In [104]:
all_current = {}

for idx in range(len(df)):
    all_current[idx] = get_current(df, idx, feature_converter=feature_converter, last_k=5)

In [105]:
all_future = {}

for idx in range(len(df)):
    all_future[idx] = get_future(df, idx, next_k=5)

In [168]:
def generate_prompt(df, idx, num_select=5):
    current_sample = all_current[idx]
    answer = all_future[idx]
    similar_samples_idxs = prompt_selector.select(current_sample, all_current, num_select=num_select)
    prompt_samples_curr = [all_current[i] for i in similar_samples_idxs]
    prompt_samples_future = [all_future[i] for i in similar_samples_idxs]
    prompt_samples = [prompt_samples_curr[i]+ " " + prompt_samples_future[i] for i in range(len(prompt_samples_curr)) \
                    if prompt_samples_future[i] is not None]
    prompt_samples = prompt_orderer.order(prompt_samples)
    prompt = prompt_samples + "\n" + current_sample
    
    next_token = ' Next:'
    next_part = answer.find(next_token)
    prompt = prompt + " "+ answer[:next_part] + next_token
    answer = answer[next_part:].lstrip(next_token)
    return prompt, answer

In [176]:
instruction = """
Given below are the details for a stock. Each line is of the following format:
Previous Candles: Candle0 Candle1 Candle2 Candle3 Candle4 Features: f1: value1 f2: value2 f3: value3 f4: value4 f5: value5 Current Candle: Candle5 Next: Candle6 Candle7 Candle8 Candle9 Candle10
Previous Candles contain the candlestick values (difference between the close and open price in the 5 minute interval) for the previous 5 timesteps. Each value contains a color (green denotes close>open and red denotes open>close) and a description of the length of the candle (how much difference was there between the open and close price).

Next, there are 5 features given for the current timestep with the description as follows:
f1: The difference between the moving averages of window sizes 1 and 10 of Opening Price - Closing Price for a stock
f2: The moving average of volume with window size 10
f3: The difference between the current closing price and the previous closing price
f4: The difference between the moving averages of window sizes 1 and 10 of Opening Price - Closing Price
f5: The length for which the slope of BBP_5_2.0 has been increasing. A negative number implies that it is decreasing

After that, the candlestick value for the current timestep is given, followed by the values of the next 5 timesteps.
There are 10 examples given below. Your goal is to predict the next 5 candlestick values.

"""

In [178]:
idx = 40
prompt, answer = generate_prompt(df, idx=idx, num_select=10)
print(instruction+prompt)
# print(answer)


Given below are the details for a stock. Each line is of the following format:
Previous Candles: Candle0 Candle1 Candle2 Candle3 Candle4 Features: f1: value1 f2: value2 f3: value3 f4: value4 f5: value5 Current Candle: Candle5 Next: Candle6 Candle7 Candle8 Candle9 Candle10
Previous Candles contain the candlestick values (difference between the close and open price in the 5 minute interval) for the previous 5 timesteps. Each value contains a color (green denotes close>open and red denotes open>close) and a description of the length of the candle (how much difference was there between the open and close price).

Next, there are 5 features given for the current timestep with the description as follows:
f1: The difference between the moving averages of window sizes 1 and 10 of Opening Price - Closing Price for a stock
f2: The moving average of volume with window size 10
f3: The difference between the current closing price and the previous closing price
f4: The difference between the moving

In [171]:
print(answer)

Green-Low, Green-Medium, Red-Medium, Red-Low, Red-Low


In [172]:
op, _, _, _ = generate(instruction+prompt, model='gpt-3.5-turbo', prompt='', max_tokens=30, temperature=0)
print(op)

Green-Low, Red-Low, Green-Medium, Red-Low, Red-Medium


In [167]:
op, _, _, _ = generate(instruction+prompt, model='text-davinci-003', prompt='', max_tokens=60, temperature=0)
print(op)

 Red-High, Green-Low, Green-High, Red-Medium, Red-Medium

Predicted Next 5 Candlestick Values: Red-High, Green-Low, Green-High, Red-Medium, Red-Medium


f1: The difference between the moving averages of window sizes 1 and 10 of Opening Price - Closing Price for a stock

f2: The moving average of volume with window size 10

f3: The difference between the current closing price and the previous closing price

f4: The difference between the moving averages of window sizes 1 and 10 of Opening Price - Closing Price

f5: The length for which the slope of BBP_5_2.0 has been increasing. A negative number implies that it is decreasing

In [ ]:
# Most similar
# Most recent
# Random

# For testing, make them into ordinal and MSE
# Do it for 5-10 samples

# Model: gpt-3.5-turbo, gpt-3 (text-davinci-003)

# Give a small description of the task, End with 'Predict the next 5'

# Run multiple times on synthetic data and average the results

# Summarize the results of the first part
# Distill the new models

In [ ]:
# Legend: 100 tokens
# Sample with Future: 40 tokens